In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [6]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [9]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,60, 100_000)
returns_scenarios = scenarios[1:3,:,:]

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
  0.115073    -0.0814348   0.01864      …   0.0554157  -0.00418046
 -0.0591554    0.043262    0.00643739      -0.0466333   0.026802
 -0.00039583   0.0036202  -0.000219411     -0.004858    0.00643925

[:, :, 3] =
 0.0348986   0.0894214    0.0804262   …  -0.0325652   -0.0303316   0.123033
 0.0593898  -0.031579    -0.0200472      -0.0612237    0.0414678  -0.0182562
 0.0193486   0.00990737   0.00276992     -0.00622268   0.0179836   0.0114965

;;; … 

[:, :, 59] =
 0.032981    0.159258    -0.00466183  …   0.0658922   -0.107958    0.0967731
 0.0681175   0.0397674    0.0379045      -0.00110814   0.0725756   0.0287192
 0.00385603  0.00968842   0.00038784      0.0061895    0.00992617  0.0131964

[:, :, 60] =
  0.00846396   0.135248     0

In [29]:
includet("kmeans_moments_lattice.jl")
using .Lattice

In [91]:
s = sum(returns_scenarios[:,:,21:25]; dims=3)
s = dropdims(s; dims= 3)

lattice = cluster(s,5)

lattice[1]

3×5 Matrix{Float64}:
 -0.0636553    0.0446905  0.16924     0.232486     -0.0754368
  0.0240154   -0.0988809  0.0619411  -0.0526456     0.140087
 -0.00214754  -0.0273481  0.0284239   0.000109913   0.0306795

In [92]:
lattice[2]

5-element Vector{Float64}:
 0.27466
 0.14795
 0.2151
 0.21133
 0.15096